In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import spacy
from nltk.corpus import state_union, stopwords
from collections import Counter

# 4.4.5 Challenge - Build Your Own NLP Model

For this challenge, you will need to choose a corpus of data from nltk or another source that includes categories you can predict and create an analysis pipeline that includes the following steps:

1. Data cleaning / processing / language parsing
2. Create features using two different NLP methods: For example, BoW vs tf-idf.
3. Use the features to fit supervised learning models for each feature set to predict the category outcomes.  Use cross-validation and determine whether one model performed better.
5. Pick one of the models and try to increase accuracy by at least 5 percentage points.

## Data Processing

In [2]:
# Import Presidential State of the Unions file ids
#state_union.fileids()

['1945-Truman.txt',
 '1946-Truman.txt',
 '1947-Truman.txt',
 '1948-Truman.txt',
 '1949-Truman.txt',
 '1950-Truman.txt',
 '1951-Truman.txt',
 '1953-Eisenhower.txt',
 '1954-Eisenhower.txt',
 '1955-Eisenhower.txt',
 '1956-Eisenhower.txt',
 '1957-Eisenhower.txt',
 '1958-Eisenhower.txt',
 '1959-Eisenhower.txt',
 '1960-Eisenhower.txt',
 '1961-Kennedy.txt',
 '1962-Kennedy.txt',
 '1963-Johnson.txt',
 '1963-Kennedy.txt',
 '1964-Johnson.txt',
 '1965-Johnson-1.txt',
 '1965-Johnson-2.txt',
 '1966-Johnson.txt',
 '1967-Johnson.txt',
 '1968-Johnson.txt',
 '1969-Johnson.txt',
 '1970-Nixon.txt',
 '1971-Nixon.txt',
 '1972-Nixon.txt',
 '1973-Nixon.txt',
 '1974-Nixon.txt',
 '1975-Ford.txt',
 '1976-Ford.txt',
 '1977-Ford.txt',
 '1978-Carter.txt',
 '1979-Carter.txt',
 '1980-Carter.txt',
 '1981-Reagan.txt',
 '1982-Reagan.txt',
 '1983-Reagan.txt',
 '1984-Reagan.txt',
 '1985-Reagan.txt',
 '1986-Reagan.txt',
 '1987-Reagan.txt',
 '1988-Reagan.txt',
 '1989-Bush.txt',
 '1990-Bush.txt',
 '1991-Bush-1.txt',
 '1991-B

In [5]:
# Grab first speech given by Bush and Clinton
clinton = state_union.raw('1993-Clinton.txt')
bush = state_union.raw('1989-Bush.txt')

In [6]:
# Parse using SpaCy
nlp = spacy.load('en')
clinton_doc = nlp(clinton)
bush_doc = nlp(bush)

In [7]:
# Group into sentences
bush_sents = [[sent, 'Bush'] for sent in bush_doc.sents]
clinton_sents = [[sent, 'Clinton'] for sent in clinton_doc.sents]

# Combine
sentences = pd.DataFrame(bush_sents + clinton_sents)
sentences.head()

,0,1
0,"(PRESIDENT, GEORGE, H.W., BUSH, 'S)",Bush
1,"(ADDRESS, ON, ADMINISTRATION, GOALS)",Bush
2,"(BEFORE, A, JOINT, SESSION, OF, CONGRESS, \n \...",Bush
3,"(Mr., Speaker, ,, Mr., President, ,, and, dist...",Bush
4,"(Less, than, 3, weeks, ago, ,, I, joined, you,...",Bush


In [8]:
# Look at excerpts from each 
print(bush_doc[:100])
print('\nBush speech length:', len(bush_doc))

print('\n', clinton_doc[:100])
print('\nClinton speech length:', len(clinton_doc))

PRESIDENT GEORGE H.W. BUSH'S ADDRESS ON ADMINISTRATION GOALS BEFORE A JOINT SESSION OF CONGRESS
 
February 9, 1989 

Mr. Speaker, Mr. President, and distinguished Members of the House and Senate, honored guests, and fellow citizens: Less than 3 weeks ago, I joined you on the West Front of this very building and, looking over the monuments to our proud past, offered you my hand in filling the next page of American history with a story of extended prosperity and continued peace. And tonight I'm back to offer

Bush speech length: 5685

 PRESIDENT BILL CLINTON'S ADDRESS BEFORE A JOINT SESSION OF THE CONGRESS ON THE STATE OF THE UNION
 
February 17, 1993 

Mr. President, Mr. Speaker, Members of the House and the Senate, distinguished Americans here as visitors in this Chamber, as am I. It is nice to have a fresh excuse for giving a long speech. [Laughter]
When Presidents speak to Congress and the Nation from this podium, typically they comment on the full range and challenges and opportunit

## Bag of Words Features

In [9]:
# Create bag of words function for each text
def bag_of_words(text):
    
    # filter out punctuation and stop words
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return most common words
    return [item[0] for item in Counter(allwords).most_common(500)]

# Get bags 
bush_words = bag_of_words(bush_doc)
clinton_words = bag_of_words(clinton_doc)

# Combine bags to create common set of unique words
common_words = set(bush_words + clinton_words)

In [10]:
# Create bag of words data frame using combined common words and sentences
def bow_features(sentences, common_words):
    
    # Build data frame
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentences in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentences
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
    
    return df

In [11]:
# Create bow features 
bow = bow_features(sentences, common_words)
bow.head()

,moral,approximately,prepared,coast,disease,rudman,spread,50,produce,angeles,...,company,punish,urgent,general,sector,party,ought,land,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(PRESIDENT, GEORGE, H.W., BUSH, 'S)",Bush
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(ADDRESS, ON, ADMINISTRATION, GOALS)",Bush
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(BEFORE, A, JOINT, SESSION, OF, CONGRESS, \n \...",Bush
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Mr., Speaker, ,, Mr., President, ,, and, dist...",Bush
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Less, than, 3, weeks, ago, ,, I, joined, you,...",Bush


## TF-IDF Features

In [26]:
# Grab sentence level documents in NLTK
clinton = state_union.sents('1993-Clinton.txt')
bush = state_union.sents('1989-Bush.txt')

In [27]:
# Create list of text 
clinton_list = [" ".join(sent) for sent in clinton]
bush_list = [" ".join(sent) for sent in bush]
joined = clinton_list + bush_list

In [31]:
# Vectorize
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.5, 
                             min_df=2, 
                             stop_words='english',   
                             use_idf=True,
                             norm=u'l2', 
                             smooth_idf=True 
                            )

tfidf = vectorizer.fit_transform(joined).tocsr()

## Supervised Learning Models
Evaluate each feature set using cross validation.  Models tested: Logistic Regression, Random Forest, & Gradient Boosting.

In [34]:
from sklearn.model_selection import cross_val_score

# Specify model inputs for each feature set

# BoW
X_bow = bow.drop(['text_sentence', 'text_source'], 1)
Y_bow = bow['text_source']

# Tfidf
X_tfidf = tfidf
Y_tfidf = ['Clinton']*len(clinton_list) + ['Bush']*len(bush_list)

### Logistic Regression

In [37]:
from sklearn.linear_model import LogisticRegression

# BoW
lr = LogisticRegression()
lr_bow = lr.fit(X_bow, Y_bow)
print('BoW Logistic Regression Scores: ', cross_val_score(lr_bow, X_bow, Y_bow, cv=5))
print('Avg Score:', np.mean(cross_val_score(lr_bow, X_bow, Y_bow, cv=5)))

# Tfidf
lr = LogisticRegression()
lr_tfidf = lr.fit(X_tfidf, Y_tfidf)
print('\nTfidf Logistic Regression Scores:', cross_val_score(lr_tfidf, X_tfidf, Y_tfidf, cv=5))
print('Avg Score:', np.mean(cross_val_score(lr_tfidf, X_tfidf, Y_tfidf, cv=5)))

BoW Logistic Regression Scores:  [0.625      0.63333333 0.65546218 0.71428571 0.72881356]
Avg Score: 0.6713789583630062

Tfidf Logistic Regression Scores: [0.58474576 0.69491525 0.66666667 0.77586207 0.76724138]
Avg Score: 0.6978862263783363


### Random Forest

In [39]:
from sklearn import ensemble

# BoW
rfc = ensemble.RandomForestClassifier()
rfc_bow = rfc.fit(X_bow, Y_bow)
print('BoW Random Forest Scores: ', cross_val_score(rfc_bow, X_bow, Y_bow, cv=5))
print('Avg Score:', np.mean(cross_val_score(rfc_bow, X_bow, Y_bow, cv=5)))

# Tfidf
rfc = ensemble.RandomForestClassifier()
rfc_tfidf = rfc.fit(X_tfidf, Y_tfidf)
print('\nTfidf Random Forest Scores:', cross_val_score(rfc_tfidf, X_tfidf, Y_tfidf, cv=5))
print('Avg Score:', np.mean(cross_val_score(rfc_tfidf, X_tfidf, Y_tfidf, cv=5)))

BoW Random Forest Scores:  [0.6        0.60833333 0.56302521 0.64705882 0.62711864]
Avg Score: 0.6325383373688459

Tfidf Random Forest Scores: [0.52542373 0.58474576 0.62393162 0.68103448 0.6637931 ]
Avg Score: 0.5967782623247263


### Gradient Boosting

In [40]:
# BoW
clf = ensemble.GradientBoostingClassifier()
clf_bow = clf.fit(X_bow, Y_bow)
print('Bow Gradient Boosting Scores:', cross_val_score(clf_bow, X_bow,Y_bow, cv=5))
print('Avg Score:', np.mean(cross_val_score(clf_bow, X_bow, Y_bow, cv=5)))

# Tfidf
clf = ensemble.GradientBoostingClassifier()
clf_tfidf = clf.fit(X_tfidf, Y_tfidf)
print('\nTfidf Random Forest Scores:', cross_val_score(clf_tfidf, X_tfidf, Y_tfidf, cv=5))
print('Avg Score:', np.mean(cross_val_score(clf_tfidf, X_tfidf, Y_tfidf, cv=5)))

Bow Gradient Boosting Scores: [0.66666667 0.625      0.67226891 0.69747899 0.6779661 ]
Avg Score: 0.654430280586811

Tfidf Random Forest Scores: [0.62711864 0.66949153 0.60683761 0.74137931 0.67241379]
Avg Score: 0.6634481759554817


### Pick A Model and Try to Increase Accuracy by 5%

__Model: Logistic Regression Using BoW Feature Set__

In [41]:
# Increase BoW size

# Update function to include 1000 most common words
def bag_of_words(text):
    
    # filter out punctuation and stop words
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return most common words
    return [item[0] for item in Counter(allwords).most_common(1000)]

# Get bags 
bush_words = bag_of_words(bush_doc)
clinton_words = bag_of_words(clinton_doc)

# Combine bags to create common set of unique words
common_words = set(bush_words + clinton_words)

In [42]:
# Create bow features 
big_bow = bow_features(sentences, common_words)

In [43]:
big_bow.head()

,relative,minimize,costly,toxic,middle,trend,reverse,important,free,"700,000",...,strength,importantly,roll,destiny,alaska,ought,constitutional,land,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(PRESIDENT, GEORGE, H.W., BUSH, 'S)",Bush
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(ADDRESS, ON, ADMINISTRATION, GOALS)",Bush
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(BEFORE, A, JOINT, SESSION, OF, CONGRESS, \n \...",Bush
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Mr., Speaker, ,, Mr., President, ,, and, dist...",Bush
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Less, than, 3, weeks, ago, ,, I, joined, you,...",Bush


In [44]:
# Make new X and Y inputs
X_big_bow = big_bow.drop(['text_sentence', 'text_source'], 1)
Y_big_bow = big_bow['text_source']

# Rerun BoW
lr = LogisticRegression()
lr_big_bow = lr.fit(X_big_bow, Y_big_bow)
print('BoW (big) Logistic Regression Scores: ', cross_val_score(lr_big_bow, X_big_bow, Y_big_bow, cv=5))
print('Avg. Score ', np.mean(cross_val_score(lr_big_bow, X_big_bow, Y_big_bow, cv=5)))

BoW (big) Logistic Regression Scores:  [0.60833333 0.625      0.63865546 0.72268908 0.72033898]
Avg. Score  0.6630033708398614


Using a bigger bag of words actually made the average score get worse by about 1%.  Try out another method - include punctuation in BoW.

In [45]:
# Update function, go back to 500 most common words and add in punctuation
def bag_of_words(text):
    
    # filter out punctuation and stop words
    allwords = [token.lemma_
                for token in text
                if not token.is_stop]
                   
    # Return most common words
    return [item[0] for item in Counter(allwords).most_common(500)]

# Get bags 
bush_words = bag_of_words(bush_doc)
clinton_words = bag_of_words(clinton_doc)

# Combine bags to create common set of unique words
common_words = set(bush_words + clinton_words)

In [46]:
# Create bow features 
bow = bow_features(sentences, common_words)

In [47]:
# Regenerate model features
X_bow = bow.drop(['text_sentence', 'text_source'], 1)
Y_bow = bow['text_source']

In [48]:
# Rerun model
lr = LogisticRegression(
    )
lr_bow = lr.fit(X_bow, Y_bow)
print('BoW #3 - Logistic Regression Scores: ', cross_val_score(lr_bow, X_bow, Y_bow, cv=5))
print('Avg. Score ', np.mean(cross_val_score(lr_bow, X_bow, Y_bow, cv=5)))

BoW #3 - Logistic Regression Scores:  [0.625      0.63333333 0.64705882 0.71428571 0.72881356]
Avg. Score  0.6696982860940986
